In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../../'))

# MedCodes Walkthrough

## Deciphering ICD Codes

International Statistical Classification of Diseases, also known as ICD, is a classification system used for patient diagnosis. ICD-9 contains 13,000 codes, while ICD-10 has 68,000 codes. Comorbidity measures, such as the Charlson comorbidity index and Elixhauser score, are used to cluster the overwhelming number of ICD codes into a smaller subset of well-defined comorbidities. 

From the `medcodes.diagnoses` module, you can use `elixhauser()` and `charlson()` to extract relevant comorbidities of a given ICD code. These functions support ICD versions 9 and 10. The code below shows how these functions work with ICD 202.61, "Malignant mast cell tumors, lymph nodes of head, face, and neck".

In [2]:
from medcodes.diagnoses import comorbidities, elixhauser, charlson

In [3]:
elixhauser(icd_code='20261', icd_version=9)

['lymphoma']

In [4]:
charlson(icd_code='20261', icd_version=9)

['malignancy']

Based on the example above, we can see that elixhauser() and charlson() have different comorbidity mappings. Elixhauser defines ICD 202.61 as "lymphoma", while Charlson maps it to "malignancy". In some cases, an ICD code may have multiple comorbidities, which is why the output is a list type.

Another useful function is `comorbidities()`, which provides a more details output of results. With `comorbidities()`, you pass in a list of ICD codes of interest, identify the version of ICD you are using, and indicate which mapping you would like. You can use "elixhauser" or "charlson" for pre-built comorbidity mappings, or "custom" if you have a specific mapping that you would like to implement.

In [5]:
icd_codes = ['1705','1364','20261','0204', '3319', '42613', '40211']

comorbidities(icd_codes, icd_version=9, mapping='elixhauser')

,icd_code,description,elixhauser_comorbidity
0,1705,Malignant neoplasm of short bones of upper limb,[solid tumor metastasis]
1,1364,Psorospermiasis,[]
2,20261,"Malignant mast cell tumors, lymph nodes of hea...",[lymphoma]
3,0204,Secondary pneumonic plague,[]
4,3319,"Cerebral degeneration, unspecified",[other neurological disorders]
5,42613,Other second degree atrioventricular block,[cardiac arrhythmias]
6,40211,Benign hypertensive heart disease with heart f...,"[congestive heart failure, hypertension, comp..."


If you decide to use a custom mapping, you will need to pass in a `custom_map` dictionary as shown in the example below. Note that by convention, ICD codes are separated with a period `.`. However, when dealing with ICD codes in medcodes, no punctuation is necessary. For example, you can use `43301` instead of `433.01`. 

In [20]:
code_mapper_v9 = {
    'ischemic stroke': ['43301', '43310', '43391', '43401'],
    'hemorrhagic stroke': ['430', '431', '432'], 
    'sepsis': ['9959','038','78552']
}

icd_codes = ['43301', '99592', '43401', '432', '430']

comorbidities(icd_codes, icd_version=9, mapping='custom', custom_map=code_mapper_v9)

,icd_code,description,custom_comorbidity
0,43301,Occlusion and stenosis of basilar artery with ...,[ischemic stroke]
1,99592,Severe sepsis,[sepsis]
2,43401,Cerebral thrombosis with cerebral infarction,[ischemic stroke]
3,432,Other and unspecified intracranial hemorrhage,[hemorrhagic stroke]
4,430,Subarachnoid hemorrhage,[hemorrhagic stroke]


## Drug Standardization and Classification

One challenge of working with clinical data is that it can be unstructured and not standardized. For example, in a clinical note, a patient's medication may be referred interchangeably as "warfarin" and "coumadin". Saline solution is often annotated as "NS". A given drug can have a wide range of names, ID's, and use cases. 

`medcodes.drugs` is a handy module for standardizing and classifying drugs. You can use the `Drug()` class to extract information for a given drug. 

In [6]:
from medcodes.drugs.standardization import Drug

In [7]:
d = Drug(drug_id='lipitor', id_type='name')

In [8]:
d.get_iupac()

'calcium;(3~{R},5~{R})-7-[2-(4-fluorophenyl)-3-phenyl-4-(phenylcarbamoyl)-5-propan-2-ylpyrrol-1-yl]-3,5-dihydroxyheptanoate'

In [9]:
d.get_inchikey()

'FQCKMBLVYCEXJB-MNSAWQCASA-L'

In [10]:
d.get_smiles()

'CC(C)C1=C(C(=C(N1CCC(CC(CC(=O)[O-])O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4.CC(C)C1=C(C(=C(N1CCC(CC(CC(=O)[O-])O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4.[Ca+2]'

In [11]:
d.describe()

Generic name: atorvastatin calcium
Brand name: Lipitor
Active ingredients: ['ATORVASTATIN CALCIUM TRIHYDRATE']
Routes of administration: ['ORAL']
Pharmacologic Classes: ['HMG-CoA Reductase Inhibitor [EPC]', 'Hydroxymethylglutaryl-CoA Reductase Inhibitors [MoA]']
NDC: 70518-0036
Product type: HUMAN PRESCRIPTION DRUG


In [12]:
d.get_pharm_class()

There are 2 pharmacologic classes.


,class_name,class_type
0,HMG-CoA Reductase Inhibitor,EPC
1,Hydroxymethylglutaryl-CoA Reductase Inhibitors,MoA


In [13]:
d.get_mesh()

,mesh_id,mesh_term
0,D000924,Anticholesteremic Agents
1,D019161,Hydroxymethylglutaryl-CoA Reductase Inhibitors


#### Drug Name Error Handling

In [15]:
Drug(drug_id='aspirinn', id_type='name')

ValueError: Drug nane not found. Here are some suggestions: ['Aspirina', 'Aspirin', 'alverine', 'alpizarin']